In [1]:
from rbc.mapd import RemoteMapD

In [2]:
omnisci = RemoteMapD()  # RemoteMapD(user=, password=, host=, port=, dbname=)

In [3]:
omnisci.sql_execute('drop table if exists mytable')
omnisci.sql_execute('create table if not exists mytable (i INT[]);');
for i in range(5):
    omnisci.sql_execute('insert into mytable values (ARRAY[1,2,3,'+str(i)+'])');
list(omnisci.sql_execute('select * from mytable')[1])

[([1, 2, 3, 0],),
 ([1, 2, 3, 1],),
 ([1, 2, 3, 2],),
 ([1, 2, 3, 3],),
 ([1, 2, 3, 4],)]

In [4]:
@omnisci('i32(i32[])')
def sum_i32(x):
    s = 0
    for i in range(len(x)):
        s += x[i]
    return s

In [6]:
descr, result = omnisci.sql_execute(
    'select i, sum_i32(i) from mytable')

list(result)

[([1, 2, 3, 0], 6),
 ([1, 2, 3, 1], 7),
 ([1, 2, 3, 2], 8),
 ([1, 2, 3, 3], 9),
 ([1, 2, 3, 4], 10)]